<a href="https://colab.research.google.com/github/gaiobr/aprendizado-de-maquina-edge-ufal/blob/master/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/



## **1. Importando bibliotecas**

In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score

from keras import preprocessing
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

## **2. Lendo imagens**

In [46]:
input_dir = '/content/drive/My Drive/Colab Notebooks/datasets/persian/'

In [66]:
#tipo = train ou test
def ler_dataset(tipo):
  X = []
  y = []
  qtd = 586 if tipo == 'train' else 200
  #para cada pasta
  for pasta in os.listdir(input_dir+tipo+'/'):
    print(pasta)
    #para cada imagem na pasta
    for img_file in os.listdir(input_dir+tipo+'/'+pasta+'/')[:qtd]:
      #leia a imagem
      img = cv2.imread(input_dir+tipo+'/'+pasta+'/'+img_file)
      #transforme imagem em array de pixels e salve na lista de imagens
      X.append(img)
      #guarde a classificação da imagem
      y.append(pasta)
  X = np.asarray(X)
  y = np.asarray(y)
  return X,y

In [67]:
X_treino,y_treino = ler_dataset('train')

18
15
11
1
17
13
16
14
10
12
9
5
4
7
6
8
2
3


In [68]:
X_treino.shape

(586,)

In [69]:
y_treino.shape

(586,)

In [70]:
X_teste,y_teste = ler_dataset('test')

1
10
11
4
3
14
18
16
2
12
13
17
15
5
6
8
7
9


In [71]:
X_teste.shape

(586,)

In [72]:
y_teste.shape

(586,)

In [73]:
X_treino

array([array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [2, 2, 2],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [4, 4, 4],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [1, 1, 1],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[4, 4, 4],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8),
       array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],

In [74]:
y_treino

array(['18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18',
       '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18',
       '18', '18', '18', '18', '18', '18', '18', '18', '18', '18', '18',
       '15', '15', '15', '15', '15', '15', '15', '15', '15', '15', '15',
       '15', '15', '15', '15', '15', '15', '15', '15', '15', '15', '15',
       '15', '15', '15', '15', '15', '15', '15', '15', '15', '15', '11',
       '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11',
       '11', '11', '11', '11', '11', '11', '11', '11', '11', '11', '11',
       '11', '11', '11', '11', '11', '11', '11', '11', '11', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '17', '17', '17', '17', '17', '17', '17',
       '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17',
       '17', '17', '17', '17', '17', '17', '17', '17',

In [75]:
#definição do modelo
model = Sequential()

# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (48,48,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 128 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

#  2 camadas fully conected 
model.add(Flatten())#converte os mapas de ativação finais para um veter de 1 dimensão
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(7, activation = "softmax"))

In [76]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [77]:


# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])



In [78]:
# Descrição do modelo
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 48, 48, 32)        2432      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 48, 48, 32)        25632     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 48, 48, 32)        25632     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 24, 24, 64)       

In [79]:
# treinamento do modelo
model.fit(X_treino, y_treino, epochs=1,verbose = 1)

ValueError: ignored